### Step 1. Download pretrained-model

In [1]:
from keras.applications import VGG16

# VGG16 was designed to work on 224 x 224 pixel input images sizes

# use 64 * 64 will train faster than 224 * 224
img_rows = 224
img_cols = 224 
s
#Loads the VGG16 model 
vgg16 = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

for layer in vgg16.layers:
    layer.trainable = False

Using TensorFlow backend.


#### check layer trainable

In [2]:
for (i,layer) in enumerate(vgg16.layers):
    print(str(i)+' '+layer.__class__.__name__,layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


## step 2. create top model and our model

In [3]:
from keras.layers import Flatten,ZeroPadding2D,Dense,Dropout
from keras.models import Model

In [4]:
def addTopModel(bottom_model, num_classes, D=256):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [5]:
num_classes = 17

FC_Head = addTopModel(vgg16, num_classes)

model = Model(inputs=vgg16.input, outputs=FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

## step 3. Load our dataset

In [6]:
from keras.preprocessing.image import ImageDataGenerator

In [7]:
train_data_dir = '../../../../../data_for_computer_vision/datasets/17_flowers/train/'
validation_data_dir = '../../../../../data_for_computer_vision/datasets/17_flowers/validation/'


train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 1190 images belonging to 17 classes.
Found 170 images belonging to 17 classes.


## step 4. Training our model

In [8]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.optimizers import RMSprop

checkpoint = ModelCheckpoint("flowers_vgg.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 1190
nb_validation_samples = 170
epochs = 5
batch_size = 16

history = model.fit(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/5
74/74 [==============================] - 26s 351ms/step - loss: 3.6930 - accuracy: 0.1533 - val_loss: 1.1579 - val_accuracy: 0.5600

Epoch 00001: val_loss improved from inf to 1.15790, saving model to flowers_vgg.h5
Epoch 2/5
74/74 [==============================] - 19s 252ms/step - loss: 2.1459 - accuracy: 0.3254 - val_loss: 0.2185 - val_accuracy: 0.6100

Epoch 00002: val_loss improved from 1.15790 to 0.21854, saving model to flowers_vgg.h5
Epoch 3/5
74/74 [==============================] - 18s 247ms/step - loss: 1.7609 - accuracy: 0.4233 - val_loss: 0.5941 - val_accuracy: 0.6200

Epoch 00003: val_loss did not improve from 0.21854
Epoch 4/5
74/74 [==============================] - 18s 241ms/step - loss: 1.5364 - accuracy: 0.4915 - val_loss: 0.8862 - val_accuracy: 0.6100

Epoch 00004: val_loss did not improve from 0.21854
Epoch 5/5
74/74 [==============================] - 17s 233ms/step - loss: 1.3939 - accuracy: 0.5443 - val_loss: 1.5647 - val_accuracy: 0.8300
Restoring mode